In [223]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas_helper import get_list_of_index_names, broadcast_simple, concat_categories

%matplotlib inline

This is just for SIDS at the moment. Can be updated for all countries when we have the data

In [224]:
#Names to WB names
any_to_wb = pd.read_csv("inputs/any_name_to_wb_name.csv",index_col="any",squeeze=True)

#GAR names with SIDS spec
gar_name_sids = pd.read_csv("inputs/gar_name_sids.csv")

#iso3 to wb country name table
iso3_to_wb=pd.read_csv("inputs/iso3_to_wb_name.csv").set_index("iso3").squeeze()

#iso2 to iso3 table
iso2_iso3 = pd.read_csv("inputs/names_to_iso.csv", usecols=["iso2","iso3"]).drop_duplicates().set_index("iso2").squeeze() #the tables has more lines than countries to account for several ways of writing country names

In [225]:
path = "sids data\\"

## Roads -- km from OSM and hard coding road costs

In [226]:
roads = pd.read_csv(path+"km_roads_sids.csv")

In [227]:
roads[['Total Length (km)', 'Total Primary',\
       'Total Secondary', 'Total Tertiary', 'Residential', 'Track / service',\
       'Total Other']] = roads[['Total Length (km)', 'Total Primary',\
       'Total Secondary', 'Total Tertiary', 'Residential', 'Track / service',\
       'Total Other']].astype(float)

In [228]:
roads["country"] = roads.country.replace(any_to_wb)

In [229]:
#cost per km of road based on st Lucia data. 
primary_road_cost = 1200000
secondary_road_cost = 700000
tertiary_road_cost = 600000

In [230]:
#reduced road values because of depreciation.
lifetime_primary = 20
lifetime_secondary = 15
lifetime_tertiary = 10

In [231]:
roads = roads.fillna(0)
roads = roads.set_index("country")

In [232]:
roads["average_age_primary"] = 8
roads["average_age_secondary"] = 5
roads["average_age_tertiary"] = 5

In [233]:
roads.loc["Kiribati","average_age_primary"]=12

In [234]:
roads["transport"] = roads["Total Primary"]*primary_road_cost*(1-1/lifetime_primary*roads.average_age_primary)+\
                             roads["Total Secondary"]*secondary_road_cost*(1-1/lifetime_secondary*roads.average_age_secondary)+\
                             roads["Total Tertiary"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)+\
                             roads["Residential"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)+\
                             roads["Track / service"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)+\
                             roads["Total Other"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)

In [235]:
roads.sample(5)

,ISO,Projection,EPSG,Sheet,Total Length (km),Total Primary,Total Secondary,Total Tertiary,Residential,Track / service,Total Other,average_age_primary,average_age_secondary,average_age_tertiary,transport
country,,,,,,,,,,,,,,,
Marshall Islands,MHL,UTM59N,32659,MHL_UTM59N,107.236929,0.000000,6.368138,8.632900,6.989726,59.002291,26.243874,8,5,5,3.323244e+07
Kiribati,KIR,UTM59N,32659,KIR_UTM59N,1190.900576,68.000000,87.000000,7.000000,67.000000,589.000000,374.000000,12,5,5,3.843400e+08
Antigua and Barbuda,ATG,UTM20N,32620,ATG_UTM20N,1368.449906,104.578230,70.561461,54.123172,757.917842,379.964498,1.304702,8,5,5,4.662181e+08
Jamaica,JAM,UTM18N,32618,JAM_UTM18N,14969.821590,1069.891646,1072.979872,1255.614165,7609.429350,1510.734627,2451.171930,8,5,5,5.119131e+09
St. Vincent and the Grenadines,VCT,UTM20N,32620,VCT_UTM20N,804.311347,62.195979,42.670103,42.014011,339.296826,164.818192,153.316237,8,5,5,2.745274e+08


## Power (from Platts)

In [236]:
power = pd.read_csv(path+"power_plants_sids.csv")
power["country"] = power.COUNTRY.apply(lambda x:x.title()).replace(any_to_wb)

In [237]:
power["power"] = power.CAPEX

## Buildings data from Rashmin and PCRAFI 

In [238]:
buildings_pcrafi = pd.read_csv(path+"PCRAFI_stocks.csv")
#values in million USD

In [239]:
#we divide by 3 because for Fiji PCRAFI is 3 times higher than Rashmin's assessment, which is consistent with GAR
buildings_pcrafi["country"] = buildings_pcrafi.country.replace(any_to_wb)
buildings_pcrafi = 1/3*10**6*buildings_pcrafi.set_index("country")

In [240]:
#buildings_pcrafi.rename(columns={"building":"total_building_value"},inplace=True)

In [241]:
buildings_pcrafi

,building,infrastructure
country,,
Cook Islands,4.322667e+08,3.920000e+07
Fiji,6.288400e+09,1.031300e+09
"Micronesia, Fed. Sts.",5.763333e+08,1.042667e+08
Kiribati,3.353667e+08,5.473333e+07
Marshall Islands,4.680333e+08,9.530000e+07
Nauru,5.793333e+07,2.466667e+07
Niue,1.368667e+08,1.400000e+07
Papua New Guinea,1.316967e+10,2.213033e+09
Palau,4.461667e+08,5.330000e+07


In [242]:
buildings_rashmin = pd.read_csv(path+"Gross_Capital_Summary.csv")
buildings_rashmin["country"] = buildings_rashmin.country.replace(iso3_to_wb).replace(any_to_wb)

In [243]:
buildings_rashmin["building"] = buildings_rashmin["Buildings (Residential + Non-Res)"]

In [244]:
buildings_rashmin.set_index("country").building

country
Belize                4.620789e+09
St. Lucia             3.197446e+09
Jamaica               3.822092e+10
Haiti                 2.656431e+10
Grenada               2.131954e+09
Dominican Republic    1.530000e+11
Fiji                  6.504790e+09
FJI est 2                      NaN
Name: building, dtype: float64

In [245]:
#Fiji has 2 estimates. first one is rashmin, last one is PCRAFI
buildings = pd.concat([buildings_rashmin.set_index("country"),buildings_pcrafi]).reset_index().drop_duplicates("country",keep="first")

### Residential vs non residential buildings

In [246]:
buildings = buildings.set_index("country")
## hard coding Fiji data from Rashmin's file Fiji_2015_exposure_estimation_110517.xlsx
fiji_total = 4093521135+2411268610
fiji_residential = 4093521135/fiji_total

# for the US, according to the BEA, residential is 19,199.1 billion and non residential 21,879.4 billion. so by default we put 50/50.
buildings["building_residential"] = buildings.building*0.5
buildings["building_non_residential"] = buildings.building*0.5
buildings.loc["Fiji","building_residential"] = buildings.loc["Fiji","building"]*fiji_residential
buildings.loc["Fiji","building_non_residential"] = buildings.loc["Fiji","building"]*(1-fiji_residential)

## Water from WASH infrastructure needs

In [247]:
water = pd.read_csv(path+"water_infra_value_sids.csv")

In [248]:
water["country"] = water["country"].replace(any_to_wb)
water.set_index("country").sample()

,water
country,
Marshall Islands,7076136.868


## Merge all available infra 

In [249]:
all_infra = pd.concat([roads.transport,power.set_index("country").power,\
                       buildings[["building_residential","building_non_residential"]],water.set_index("country")],axis=1)

In [250]:
all_infra["all_infra_but_transport"] = all_infra.power+all_infra.building_non_residential+all_infra.building_residential+all_infra.water

In [251]:
all_infra.sample(3)

,transport,power,building_residential,building_non_residential,water,all_infra_but_transport
Cape Verde,NaN,112996443.0,NaN,NaN,NaN,NaN
Antigua and Barbuda,4.662181e+08,43136405.0,NaN,NaN,NaN,NaN
Marshall Islands,3.323244e+07,15285248.0,2.340167e+08,2.340167e+08,7076136.868,4.903947e+08


In [252]:
all_infra = all_infra[["transport","all_infra_but_transport"]].dropna().unstack().reset_index().rename(columns={"level_0":"sector","level_1":"country",0:"infra_k"}).set_index(["country","sector"])

In [253]:
all_infra

,,infra_k
country,sector,
Belize,transport,2.513105e+09
Fiji,transport,2.981647e+09
Haiti,transport,9.742686e+09
Jamaica,transport,5.119131e+09
Kiribati,transport,3.843400e+08
Marshall Islands,transport,3.323244e+07
"Micronesia, Fed. Sts.",transport,1.575350e+08
Papua New Guinea,transport,6.976123e+09
Samoa,transport,9.025303e+08


In [254]:
all_infra.sum(level="country")

,infra_k
country,
Belize,7.815936e+09
Fiji,1.038296e+10
Haiti,3.780226e+10
Jamaica,4.838888e+10
Kiribati,7.355168e+08
Marshall Islands,5.236272e+08
"Micronesia, Fed. Sts.",7.702267e+08
Papua New Guinea,2.188859e+10
Samoa,1.737938e+09


shares = all_infra.dropna()
shares = shares.multiply(1/shares.total,axis="index").drop("total",axis=1)

shares.rename(columns={'total_roads_value':"roads", 'total_power_value':"power plants", 'total_building_value':"buildings", 'total_water_value':"water"}).plot(kind="bar",stacked=True)
plt.legend(bbox_to_anchor=(1.05, 1.0))
plt.ylabel("Share in total infrastructure stock")
plt.savefig("infra_share.png",bbox_inches='tight')

## Read total capital data gathered in "gather_capital_data" and wb data gathered in "download_wb_data"

In [255]:
K = pd.read_csv("intermediate/avg_prod_k_with_gar_for_sids.csv",index_col="Unnamed: 0")
wb = pd.read_csv("inputs/wb_data.csv",index_col="country")

## recomputes Ktot from WB GDP and average capital productivity

In [256]:
wb["Ktot"] = wb.gdp_pc_pp*wb['pop']/K.avg_prod_k
wb["GDP"] = wb.gdp_pc_pp*wb['pop']

In [257]:
all_infra[['Ktot','GDP']] = broadcast_simple(wb[['Ktot','GDP']],all_infra.index)
all_infra

,,infra_k,Ktot,GDP
country,sector,,,
Belize,transport,2.513105e+09,1.073572e+10,2.873784e+09
Fiji,transport,2.981647e+09,2.288247e+10,7.965477e+09
Haiti,transport,9.742686e+09,1.345242e+11,1.794108e+10
Jamaica,transport,5.119131e+09,1.161836e+11,2.359830e+10
Kiribati,transport,3.843400e+08,7.645732e+08,2.171002e+08
Marshall Islands,transport,3.323244e+07,NaN,2.003310e+08
"Micronesia, Fed. Sts.",transport,1.575350e+08,NaN,3.495423e+08
Papua New Guinea,transport,6.976123e+09,6.516308e+10,2.118981e+10
Samoa,transport,9.025303e+08,NaN,1.147754e+09


In [258]:
all_infra["share"] = all_infra.infra_k/all_infra.Ktot

sids_infra["share_transport_in_K"] = sids_infra.total_roads_value/sids_infra.Ktot
sids_infra["share_transport_infra_but_buildings"] = sids_infra.total_roads_value/(sids_infra.total_power_value+sids_infra.total_roads_value+sids_infra.total_water_value)

sids_infra.dropna(subset=["share_transport_in_K"]).to_csv("for_sids_report.csv")

In [259]:
SIDS_infra = all_infra.dropna().share.unstack("sector").assign(other_k=lambda df: 1 - df.transport - df.all_infra_but_transport).stack()
SIDS_infra.name="share"

In [262]:
pd.DataFrame(SIDS_infra).to_csv("inputs/sids_infra_stocks.csv")

In [261]:
pd.DataFrame(SIDS_infra)

share
country          sector                           
Belize           all_infra_but_transport  0.493943
                 transport                0.234088
                 other_k                  0.271969
Fiji             all_infra_but_transport  0.323449
                 transport                0.130303
                 other_k                  0.546248
Haiti            all_infra_but_transport  0.208584
                 transport                0.072423
                 other_k                  0.718993
Jamaica          all_infra_but_transport  0.372426
                 transport                0.044061
                 other_k                  0.583514
Kiribati         all_infra_but_transport  0.459311
                 transport                0.502686
                 other_k                  0.038003
Papua New Guinea all_infra_but_transport  0.228848
                 transport                0.107056
                 other_k                  0.664095
Solomon Islands  all_infra_but_transport  0.262207
                 transport                0.141125
                 other_k                  0.596668
St. Lucia        all_infra_but_transport  0.695867
                 transport                0.135446
                 other_k                  0.168686
Tonga            all_infra_but_transport  0.557667
                 transport                0.359542
                 other_k                  0.082791